In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
data = pd.read_csv('../data/combined_data_v6.csv')
data['headline'] = data['headline'].astype('U')
data['year'] = data['year'].astype(int)
data = data.drop_duplicates().reset_index(drop = True)
data.shape

(439412, 5)

In [5]:
data.sample(n = 10)

,source,headline,year,date,is_sarcastic
45871,Huffington Post,Firefighters Battle Major Building Fire In Oak...,2017,2017-03-27,0
368141,atlantic,The Bipartisan Opposition to Sessions's New Ci...,2017,2017-07-19,0
386697,onion,Uber Pressured To Allow Tipping,2017,2017-04-20,1
422239,Fox News,Hope Solo adds to US women's soccer celebratio...,2019,2019-06-13,0
314993,guardian,How did America become a pariah nation of supe...,2020,2020-06-26,0
393121,onion,Brewers Players Showering Themselves In Beer U...,2019,2019-09-27,1
276046,npr,Trump Thanks Putin For Ordering Expulsion Of D...,2017,2017-08-10,0
124883,ABC Australia,adelaide prisoner shackled to hospital bed for...,2016,2016-08-12,0
46739,Huffington Post,Chrissy Teigen And Baby Luna Wear The Cutest M...,2017,2017-03-15,0
339984,guardian,There must be easier fads to follow than teach...,2018,2018-02-15,0


### Preprocessing Steps
1. Standardize common abbreviations* (WIP)
    1. u.s. --> usa
1. Lowercase
1. Expand Contractions
1. Optional:
    1. Remove Source Specific Language
    1. Remove Profanity
1. Remove Special Characters


In [6]:
def replace_words(text, replace_dict):
    tokens = []
    for w in text.split():
        word = w
        for t in replace_dict.keys():
            if w == t:
                word = replace_dict[t]
        tokens.append(word)
    
    return " ".join(tokens)

In [7]:
data['clean'] = data['headline']

## Standardize Common Abbreviations
translate_dict = {
    "US": "USA",
    "U.S.": "USA",
    "u.s.": "USA",
    "u.s": "USA",
}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, translate_dict))

In [8]:
# Lowercase
data['clean'] = data['clean'].apply(lambda x: x.lower())

In [9]:
# Expand Contractions
contractions_dict = { "ain't": "are not","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who's": "who is",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, contractions_dict))


In [10]:
data.loc[data['headline'].str.contains("'ll"), ['headline','clean']]

,headline,clean
170,6 summer salads you'll actually crave,6 summer salads you will actually crave
190,20 suede pieces you'll want to wear all spring,20 suede pieces you will want to wear all spring
581,you'll want to get inked after seeing this tat...,you will want to get inked after seeing this t...
621,the hilarious hipster classifieds you'll (prob...,the hilarious hipster classifieds you will (pr...
721,"apple fritter season is here, and so are the r...","apple fritter season is here, and so are the r..."
...,...,...
437925,Maryland authorities say they'll investigate K...,maryland authorities say they will investigate...
438361,Elizabeth Warren says she'll wear Planned Pare...,elizabeth warren says she will wear planned pa...
438628,Mom had heart attack while driving kids to sch...,mom had heart attack while driving kids to sch...
438647,Paula White on how she became Trump's spiritua...,paula white on how she became trump's spiritua...


In [11]:
### Remove bad text:

def remove_artifacts(s):
    return re.sub('xe[0-9]+x[0-9]+x[0-9]+', '',s)

data['clean'] = data['clean'].apply(lambda x: remove_artifacts(x))

In [12]:
data.loc[data['headline'].str.contains("xe"), ['headline','clean']]

,headline,clean
24,drug-resistant bacteria often lurk in children...,drug-resistant bacteria often lurk in children...
220,arkansas executes first inmate in 12 years,arkansas executes first inmate in 12 years
356,several women accuse progressive media executi...,several women accuse progressive media executi...
620,do you have to pay income taxes on social secu...,do you have to pay income taxes on social secu...
845,"grab your wine boxes, because 'will & grace' i...","grab your wine boxes, because 'will & grace' i..."
...,...,...
439369,"Wildfires threaten firefighters, homes across ...","wildfires threaten firefighters, homes across ..."
439377,Biden tells Pelosi and Schumer xe2x80x98mi cas...,"biden tells pelosi and schumer mi casa, you casa"
439378,Director insists xe2x80x98The Young Messiahxe2...,director insists the young messiah does not co...
439395,Brett Young shares hilarious story about wifex...,brett young shares hilarious story about wifes...


In [13]:
## Remove Special Characters
data['clean'] = data['clean'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))

In [14]:
## Trim Extra White Space
text = data['clean'].apply(lambda x: re.sub('\s{2,}', ' ', x))

In [15]:
guardian_length = data[data.source == 'guardian'].shape[0]

Huff = data[data.source == 'Huffington Post']
ABC = data[data.source == 'ABC Australia']

other_sources = data[(data.source != 'Huffington Post') & (data.source != 'ABC Australia')]

Huff = Huff.sample(n=guardian_length)
ABC = ABC.sample(n=guardian_length)

data_sources = [Huff, ABC, other_sources]
data = pd.concat(data_sources)
data.reset_index(drop=True, inplace = True)
data.shape

(246667, 6)

In [16]:
data[['source', 'year', 'clean', 'is_sarcastic']].to_csv('../data/combined_clean_v8.csv', index = False, sep = "|")

In [17]:
# df = data[['source', 'year', 'clean', 'is_sarcastic']]
# df.columns = ['source','year','headline','is_sarcastic']

In [18]:
# df.to_csv('../data/combined_data_v4.csv',index = False)